In [ ]:
#作ったモデルを動かしてみる
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
def perplexity(model, tokenizer, text) -> torch.Tensor:
    tokenized_input = tokenizer.encode(
        text, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)
    with torch.inference_mode():
        output = model(tokenized_input, labels=tokenized_input)
    ppl = torch.exp(output.loss)
    return ppl.item()


In [ ]:

model_path="../../models/hf/1_step30800"
#model_path= "llm-jp/llm-jp-13b-v1.0"

model_dict={
"llmjp13b":"llm-jp/llm-jp-13b-v1.0",
"1_step30800":"../../models/hf/1_step30800",
"2_step37800":"../../models/hf/2_step37800",
"3_step43400":"../../models/hf/3_step43400",
"4_step49700":"../../models/hf/4_step49700",
"5_step58800":"../../models/hf/5_step58800",
}


In [ ]:
model_path=model_dict["5_step58800"]

In [ ]:

def init_pipe(model_name):
    model_path=model_dict[model_name]
    model = AutoModelForCausalLM.from_pretrained(model_path,device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, 
                max_new_tokens=100, 
                repetition_penalty=1.2,
                temperature=0.7,
                )
    return model,tokenizer,pipe

In [ ]:
import time

text_list=[
"富士山は",
"化学研究の問題点は",
"Functional polymers are",
"Bioinformatics is",
"フィボナッチ数列を生成するPythonのコードは次のとおりです｡\n def",
"ホンダ フィット 販売",
"英語: He is a good man. 日本語: ",
"英語: What is the problem of developing LLM in Japan? 日本語: ",
"Q: 日本の首都は? A:",
"Q: ドラえもんの友達は? A:",
"Q: 純粋理性批判はたぬきに理解できますか? A:",
"Q: ChatGPTとはなんですか? A:",
"Q: 俳句を書いてください A:",
"Q: 日本の総理大臣は? A:",
]

out_dict={}

for model_name in model_dict.keys():
    print(model_name)
    model,tokenizer,pipe=init_pipe(model_name)
    model_output_dict={}
    for text in text_list:
        perp=perplexity(model,tokenizer,text)
        s_time=time.time()
        res=pipe(text)[0]["generated_text"]
        consumed_time=time.time()-s_time
        model_output_dict[text]={
            "perplexity":perp,
            "output":res
        }
        print("-------")
        print("input: ", text.replace("\n",""))
        print("perplexity: ",perp)
        print("time: ", consumed_time)
        print("time/character: ", consumed_time/len(res))
        print("output: ",res.replace("\n",""))
    out_dict[model_name]=model_output_dict

In [ ]:
import json
#with open("test/output.json","w") as f:
#    json.dump(out_dict,f,indent=4,ensure_ascii=False)

In [ ]:
import pandas as pd
df=pd.DataFrame(out_dict)

perp_dict={}
text_dict={}
for model_name in model_dict.keys():
    temp_dict={}
    temp_dict2={}
    for text in text_list:
        temp_dict[text]=out_dict[model_name][text]["perplexity"]
        temp_dict2[text]=out_dict[model_name][text]["output"][len(text):]
    perp_dict[model_name]=temp_dict
    text_dict[model_name]=temp_dict2

df_perp=pd.DataFrame(perp_dict)
df_perp

In [ ]:
df_text=pd.DataFrame(text_dict)
df_text